---
Detecting jump pose in a video using Mediapipe
---
<h3>Author : Anoop Reddy Kallem
<h3>Date : 09-04-2024
<h3>Program/Branch : B.Tech Computer Science and Engineering (Specialisation in Data Science)

In [ ]:
!pip install cv2
!pip install mediapipe


In [2]:
import cv2
import mediapipe as mp
import numpy as np
from time import time



**Initialising Mediapose class**


In [3]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3, model_complexity=2)

**Initialise jump detection variables**

In [4]:
last_ankle_position = None
jump_threshold = 50 

**Read video file using `cv2.VideoCapture(videopath)`**

In [5]:
video_path = 'video_test.mov'
video = cv2.VideoCapture(video_path)

**Creating a normal CV2 window so that there is no automatic cropping and zoom-ins and create a variable to store the drawing/tracking utils from MediaPipe**

In [6]:
cv2.namedWindow('Jump Detection', cv2.WINDOW_NORMAL)
mp_drawing = mp.solutions.drawing_utils

**Loop through frames**
- Preprocess the video from BGR to RGB since cv2 only considers the input to be in (r,g,b) format.
- Check if the Pose Landmark is found.
- Declare and compute the Left and Right ankle via `results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE]` and `results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE]`.
- Check if the current frame is first frame or there are previous ankle positions to compare with.
- Compute vertical displacement of **current** ankle positions and **previous** ankle positions.
- If the difference is greater than the `jump_threshold` then `DETECTED` will be annotated the top right c corner of the frame in bright green colour.
- Update last ankle position with the current y-coordinate of the left and right ankles for using in the next iteration.
- Draw the body points to track the pose.
- Repeat or break the loop when `q` is pressed. 

In [7]:
while video.isOpened():
    
    ret, frame = video.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.pose_landmarks:
        left_ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE]
        right_ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE]

        if last_ankle_position is not None:
            left_displacement = abs(left_ankle.y * frame.shape[0] - last_ankle_position[0])
            right_displacement = abs(right_ankle.y * frame.shape[0] - last_ankle_position[1])

            if left_displacement > jump_threshold or right_displacement > jump_threshold:
                cv2.putText(frame, 'DETECTED', (frame.shape[1] - 700, 90), cv2.FONT_HERSHEY_DUPLEX, 4, (0, 255, 0), 10, cv2.LINE_AA)
        last_ankle_position = (left_ankle.y * frame.shape[0], right_ankle.y * frame.shape[0])
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow('Jump Detection', frame)


    key = cv2.waitKey(1)
    if key == ord('q'):
        break

**Release the `VideoCapture` object and destroy all the windows before the program exits**

In [8]:
video.release()
cv2.destroyAllWindows()